In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.preprocessing import image 
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [8]:
import cartoon_landmarks as ca

In [19]:
def get_data():

    X, y = extract_features_labels()
#     Y = np.array([y, -(y - 1)]).T
#     tr_X = X[:100]
#     tr_Y = Y[:100]
#     te_X = X[100:]
#     te_Y = Y[100:]
    tr_X, te_X, tr_Y, te_Y = train_test_split(X, y,random_state=0)

    return tr_X, tr_Y, te_X, te_Y

In [3]:
#
name = ['eye_color', 'face_shape']
#data = pd.read_csv('./dataset_AMLS_20-21/celeba/labels.csv',sep='\t',header=0,names = name)
data = pd.read_csv('./dataset_AMLS_20-21/cartoon_set/labels.csv',sep='\t')
Y_A10=data['eye_color']
Y_A11=data['face_shape']

In [4]:
#
#图片读取
train_image = [] 
for i in tqdm(range(data.shape[0])):
    #img = image.load_img('./dataset_AMLS_20-21/celeba/img/'+str(i)+'.jpg', color_mode='rgb', target_size=None, grayscale=False)
    img = image.load_img('./dataset_AMLS_20-21/cartoon_set/img/'+str(i)+'.png', target_size=None, grayscale=True)
    img = image.img_to_array(img)     
    img = img/255     
    train_image.append(img) 
X = np.array(train_image)

100%|██████████| 10000/10000 [00:46<00:00, 216.86it/s]


In [5]:
#
X, Y_A10 = shuffle(X,Y_A10)
x_train, x_test, y_train, y_test = train_test_split(X, Y_A10,random_state=0)

In [17]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn import svm
from sklearn.svm import SVC

In [13]:
import os
import numpy as np
from keras.preprocessing import image
import cv2
import dlib
import pandas as pd
from pathlib import PurePath
#import defusedxml

# PATH TO ALL IMAGES
global basedir, image_paths, target_size
#basedir = './AMLS_assignment20_21-/dataset_AMLS_20-21'
basedir = './dataset_AMLS_20-21'
basedir = os.path.join(basedir,'cartoon_set')
#images_dir = basedir
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

# def listToString(s):
#     str1 = " "
#     return (str1.join(s))

def extract_features_labels():
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extracts the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        gender_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """
    #print(images_dir)
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    target_size = None
    
    # name = ['eye_color', 'face_shape']
    #data = pd.read_csv('./dataset_AMLS_20-21/celeba/labels.csv',sep='\t',header=0,names = name)
    data = pd.read_csv(os.path.join(basedir, labels_filename),sep='\t')
#     Y_A10=data['gender']
#     Y_A11=data['smiling']
#     y_1 = Y_A10
#     y_2 = Y_A11
    eye = data['eye_color']
    faceshape = data['face_shape']
    #print(gender_labels[0:5])

#     labels_file = open(os.path.join(basedir, labels_filename), 'r')
#     lines = labels_file.readlines() 
# # from list get str get array symbols
# for i in range(len(rows)):
#     str0 = listToString(rows[i])
#     y = str0.split('\t')
#     y = np.array(y)
#     Y_A1[i,0] = y[2]
#     Y_A1[i,1] = y[3]
#     #str = str+str0
    #gender_labels = {listToString(line).split('\t')[0] : int(listToString(line).split('\t')[3]) for line in lines[1:]}
    #gender_labels = {listToString(line).split('\t')[3] for line in lines[1:]}

#test    
#     for line in lines[1:10]:
#         #print(line)
#         gender_labels.append(listToString(line).split('\t')[2])
#         a = listToString(line).split('\t')
#         print(np.array(a)[2])
#     #gender_labels = np.array(gender_labels)
                      
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            #print(img_path)
            file_name= PurePath(img_path.split('.')[1]).parts[-1]
            file_name=int(file_name)
            #print(file_name)
            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            #print(img)
            features, _ = run_dlib_shape(img)
            #print(features)
            if features is not None:
                all_features.append(features)
                #print(gender_labels[file_name])
                all_labels.append(eye[file_name])
                #all_labels.append(gender_labels[file_name])
    #print(all_labels)

    landmark_features = np.array(all_features)
#     labels = (np.array(all_labels, dtype=float) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
    labels = np.array(all_labels)
    return landmark_features, labels

In [20]:
tr_X, tr_Y, te_X, te_Y= get_data()
print(tr_X.shape, te_X.shape)

(6146, 68, 2) (2049, 68, 2)


In [28]:
tr_X = tr_X/tr_X.max()
te_X = te_X/te_X.max()
print(tr_Y[0:5])
print(tr_X[0,:,:])

[1 4 3 1 0]
[[0.34907598 0.53798768]
 [0.35318275 0.58521561]
 [0.3613963  0.63039014]
 [0.37577002 0.67351129]
 [0.3963039  0.71457906]
 [0.42299795 0.74537988]
 [0.45379877 0.77412731]
 [0.4825462  0.79671458]
 [0.51129363 0.80698152]
 [0.53798768 0.80082136]
 [0.5687885  0.7761807 ]
 [0.60164271 0.74537988]
 [0.62833676 0.71252567]
 [0.65092402 0.67145791]
 [0.66324435 0.62833676]
 [0.66940452 0.58726899]
 [0.67351129 0.54414784]
 [0.38603696 0.49691992]
 [0.40451745 0.4825462 ]
 [0.42915811 0.47433265]
 [0.45585216 0.48049281]
 [0.47638604 0.49486653]
 [0.5523614  0.49281314]
 [0.57494867 0.4825462 ]
 [0.59958932 0.47843943]
 [0.62422998 0.48665298]
 [0.6386037  0.49897331]
 [0.51540041 0.52977413]
 [0.51540041 0.56057495]
 [0.51540041 0.59137577]
 [0.51540041 0.62422998]
 [0.4825462  0.63244353]
 [0.49691992 0.6386037 ]
 [0.51540041 0.64476386]
 [0.52977413 0.64065708]
 [0.54414784 0.63244353]
 [0.41889117 0.53593429]
 [0.43531828 0.52566735]
 [0.45174538 0.52772074]
 [0.46817248 

In [37]:
# sklearn functions implementation
def img_SVM(training_images, training_labels, test_images, test_labels):
    classifier = svm.LinearSVC(penalty='l1', dual=False, multi_class='ovr', random_state=None, max_iter=10000)
#     LinearSVC(penalty='l2', loss='squared_hinge', *, dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)
    classifier.fit(training_images, training_labels)
    pred = classifier.predict(test_images)
    print("Accuracy:", accuracy_score(test_labels, pred))

   # print(pred)
    return pred

In [38]:
#svm.LinearSVC()：Accuracy: 0.21473889702293802,failed to converge；max_iter=5000:Accuracy: 0.21815519765739386failed converge= =；
#图像归一化后max_iter=5000&10000:Accuracy: 0.3430941922889214 converge;
# pred = img_SVM(x_train.reshape((x_train.shape[0], x_train.shape[1]*x_train.shape[2])), y_train, x_test.reshape((x_test.shape[0], x_test.shape[1]*x_test.shape[2])), y_test)
pred = img_SVM(tr_X.reshape(6146,136), tr_Y, te_X.reshape(2049,136), te_Y)

Accuracy: 0.3504148365056125


C:\Users\93554\anaconda3\envs\mytf\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
